In [3]:
import requests
from pprint import pprint
import os



APP_ID = os.environ.get("TFL_APP_ID")
APP_ID = "Halfway"
APP_KEY = os.environ.get("TFL_APP_KEY")
APP_KEY = ""
base_url = "https://api.tfl.gov.uk"


def fetch_tfl_data(endpoint, params=None):
    url = f"{base_url}{endpoint}"
    params = params or {}
    params.update({"app_id": APP_ID, "app_key": APP_KEY})

    try:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"\nHTTP Error {response.status_code}: {response.text}")
        return None
    except Exception as e:
        print(f"\nError fetching {url}: {str(e)}")
        return None

In [4]:
modes = ["tube", "overground", "dlr", "elizabeth-line"]
lines = fetch_tfl_data("/Line/Mode/" + ",".join(modes))

In [5]:
line_ids = []
for line in lines:
    print(line["id"], line["name"], line["modeName"])
    line_ids.append(line["id"])


bakerloo Bakerloo tube
central Central tube
circle Circle tube
district District tube
dlr DLR dlr
elizabeth Elizabeth line elizabeth-line
hammersmith-city Hammersmith & City tube
jubilee Jubilee tube
liberty Liberty overground
lioness Lioness overground
metropolitan Metropolitan tube
mildmay Mildmay overground
northern Northern tube
piccadilly Piccadilly tube
suffragette Suffragette overground
victoria Victoria tube
waterloo-city Waterloo & City tube
weaver Weaver overground
windrush Windrush overground


In [47]:
station_id = {}

for line_id in line_ids:
    stops = fetch_tfl_data(f"/Line/{line_id}/StopPoints")
    #print(stops[0].keys())
    #pprint(stops[1], depth=1, compact=True)
    #print("\n\n")
    #pprint(stops[0]["children"])
    for stop in stops:
        name = stop["commonName"]
        naptanId = stop["naptanId"]
        print(f"{naptanId}: {name} - {line_id}")
        station_id[naptanId] = name

940GZZLUBST: Baker Street Underground Station - bakerloo
940GZZLUCHX: Charing Cross Underground Station - bakerloo
940GZZLUEAC: Elephant & Castle Underground Station - bakerloo
940GZZLUEMB: Embankment Underground Station - bakerloo
940GZZLUERB: Edgware Road (Bakerloo) Underground Station - bakerloo
940GZZLUHAW: Harrow & Wealdstone Underground Station - bakerloo
940GZZLUHSN: Harlesden Underground Station - bakerloo
940GZZLUKEN: Kenton Underground Station - bakerloo
940GZZLUKPK: Kilburn Park Underground Station - bakerloo
940GZZLUKSL: Kensal Green Underground Station - bakerloo
940GZZLULBN: Lambeth North Underground Station - bakerloo
940GZZLUMVL: Maida Vale Underground Station - bakerloo
940GZZLUMYB: Marylebone Underground Station - bakerloo
940GZZLUNWY: North Wembley Underground Station - bakerloo
940GZZLUOXC: Oxford Circus Underground Station - bakerloo
940GZZLUPAC: Paddington Underground Station - bakerloo
940GZZLUPCC: Piccadilly Circus Underground Station - bakerloo
940GZZLUQPS: Que

In [51]:
id = "northern"
direction = "outbound"
stops = fetch_tfl_data(f"/Line/{id}/Route/Sequence/{direction}")

In [54]:
for station in stops["orderedLineRoutes"][2]["naptanIds"]: # Routes with variations have multiple  entries here
    print(f"{id}: {station_id[station]}")

northern: Morden Underground Station
northern: South Wimbledon Underground Station
northern: Colliers Wood Underground Station
northern: Tooting Broadway Underground Station
northern: Tooting Bec Underground Station
northern: Balham Underground Station
northern: Clapham South Underground Station
northern: Clapham Common Underground Station
northern: Clapham North Underground Station
northern: Stockwell Underground Station
northern: Oval Underground Station
northern: Kennington Underground Station
northern: Elephant & Castle Underground Station
northern: Borough Underground Station
northern: London Bridge Underground Station
northern: Bank Underground Station
northern: Moorgate Underground Station
northern: Old Street Underground Station
northern: Angel Underground Station
northern: King's Cross St. Pancras Underground Station
northern: Euston Underground Station
northern: Camden Town Underground Station
northern: Chalk Farm Underground Station
northern: Belsize Park Underground Station

In [36]:
#pprint(stops, depth=1, compact=True)
pprint(stops["orderedLineRoutes"][0], depth=2, compact=True)

{'$type': 'Tfl.Api.Presentation.Entities.OrderedRoute, '
          'Tfl.Api.Presentation.Entities',
 'name': 'Brixton  &harr;  Walthamstow Central ',
 'naptanIds': ['940GZZLUBXN', '940GZZLUSKW', '940GZZLUVXL', '940GZZLUPCO',
               '940GZZLUVIC', '940GZZLUGPK', '940GZZLUOXC', '940GZZLUWRR',
               '940GZZLUEUS', '940GZZLUKSX', '940GZZLUHAI', '940GZZLUFPK',
               '940GZZLUSVS', '940GZZLUTMH', '940GZZLUBLR', '940GZZLUWWL'],
 'serviceType': 'Regular'}


In [56]:
id = "victoria"
fromStopPointId = "940GZZLUBXN"
timetable = fetch_tfl_data(f"/Line/{id}/Timetable/{fromStopPointId}")

In [57]:
pprint(timetable, depth=1)

{'$type': 'Tfl.Api.Presentation.Entities.TimetableResponse, '
          'Tfl.Api.Presentation.Entities',
 'direction': 'outbound',
 'lineId': 'victoria',
 'lineName': 'Victoria',
 'stations': [...],
 'stops': [...],
 'timetable': {...}}


In [ ]:
pprint(timetable["timetable"]["routes"][0]["schedules"][0])

In [80]:

def time_from_journey(journey):
    return int(journey["hour"]) * 60 + int(journey["minute"])
    
first = timetable["timetable"]["routes"][0]["schedules"][0]["firstJourney"]
first_time = time_from_journey(first)
for schedule in timetable["timetable"]["routes"][0]["schedules"][0]["knownJourneys"][1:]:
    next_time = time_from_journey(schedule)
    print(f"Time between Trains = {next_time - first_time} mins")
    first_time = next_time

    

Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 10 mins
Time between Trains = 9 mins
Time between Trains = 8 mins
Time between Trains = 6 mins
Time between Trains = 5 mins
Time between Trains = 5 mins
Time between Trains = 4 mins
Time between Trains = 5 mins
Time between Trains = 4 mins
Time between Trains = 4 mins
Time between Trains = 4 mins
Time between Trains = 3 mins
Time between Trains = 4 mins
Time between Trains = 3 mins
Time between Trains = 2 mins
Time between Trains = 3 mins
Time between Trains = 3 mins
Time between Trains = 2 mins
Time between Trains = 3 mins
Time between Trains = 3 mins
Time between Trains = 2 mins


In [91]:
pprint(timetable["timetable"]["routes"][0]["stationIntervals"], depth=2)

[{'$type': 'Tfl.Api.Presentation.Entities.StationInterval, '
           'Tfl.Api.Presentation.Entities',
  'id': '0',
  'intervals': [...]},
 {'$type': 'Tfl.Api.Presentation.Entities.StationInterval, '
           'Tfl.Api.Presentation.Entities',
  'id': '1',
  'intervals': [...]}]


In [93]:
for stop in timetable["timetable"]["routes"][0]["stationIntervals"][0]["intervals"]:
    name = station_id[stop["stopId"]]
    arrivalTime = stop["timeToArrival"]
    print(f"{name}: {arrivalTime}") # Why are there 2 for almost all stations?

Stockwell Underground Station: 1.0
Stockwell Underground Station: 2.0
Vauxhall Underground Station: 3.0
Vauxhall Underground Station: 4.0
Pimlico Underground Station: 5.0
Pimlico Underground Station: 5.0
Victoria Underground Station: 7.0
Victoria Underground Station: 7.0
Green Park Underground Station: 9.0
Green Park Underground Station: 9.0
Oxford Circus Underground Station: 11.0
Oxford Circus Underground Station: 11.0
Warren Street Underground Station: 12.0
Warren Street Underground Station: 13.0
Euston Underground Station: 14.0
Euston Underground Station: 14.0
King's Cross St. Pancras Underground Station: 15.0
King's Cross St. Pancras Underground Station: 16.0
Highbury & Islington Underground Station: 18.0
Highbury & Islington Underground Station: 19.0
Finsbury Park Underground Station: 21.0
Finsbury Park Underground Station: 21.0
Seven Sisters Underground Station: 24.0
Seven Sisters Underground Station: 25.0
Tottenham Hale Underground Station: 26.0
Tottenham Hale Underground Statio

In [94]:
for stop in timetable["timetable"]["routes"][0]["stationIntervals"][1]["intervals"]:
    name = station_id[stop["stopId"]]
    arrivalTime = stop["timeToArrival"]
    print(f"{name}: {arrivalTime}") # Why are there 2 for almost all stations?

Stockwell Underground Station: 1.0
Stockwell Underground Station: 2.0
Vauxhall Underground Station: 3.0
Vauxhall Underground Station: 4.0
Pimlico Underground Station: 5.0
Pimlico Underground Station: 5.0
Victoria Underground Station: 7.0
Victoria Underground Station: 7.0
Green Park Underground Station: 9.0
Green Park Underground Station: 9.0
Oxford Circus Underground Station: 11.0
Oxford Circus Underground Station: 11.0
Warren Street Underground Station: 12.0
Warren Street Underground Station: 13.0
Euston Underground Station: 14.0
Euston Underground Station: 14.0
King's Cross St. Pancras Underground Station: 15.0
King's Cross St. Pancras Underground Station: 16.0
Highbury & Islington Underground Station: 18.0
Highbury & Islington Underground Station: 19.0
Finsbury Park Underground Station: 21.0
Finsbury Park Underground Station: 21.0
Seven Sisters Underground Station: 25.0
